# Definição de funções e imports necessários

In [5]:
import pandas as pd
import numpy as np
import json
import pymongo

### Funções

In [6]:
def readXLSX (path):
    return pd.read_excel(path, index_col=None);

def openDataBaseMongoDB ():
    client = pymongo.MongoClient("mongodb://<ADMIN>:<PASSWORD>@criedata-shard-00-00.urzuu.mongodb.net:27017,criedata-shard-00-01.urzuu.mongodb.net:27017,criedata-shard-00-02.urzuu.mongodb.net:27017/<dbname>?ssl=true&replicaSet=atlas-6db9id-shard-0&authSource=admin&retryWrites=true&w=majority");
    db = client.schoolData;
    return db.dataINEPSchools;

def deletingColumns (dataframe):
    columns = ["Região", 
               "UF", 
               "Código do Município", 
               "Nome do Município", 
               "Nome da Escola", 
               "Localização", 
               "Dependência Administrativa"];
    
    for column in columns:
        try:
            dataframe.pop(column)
        except:
            continue
    
def JSONstructure (data):    
    dataJSON = {"Percentual de funções docentes com curso superior": 
                {
                "Última atualização": data["Ano"],
                "Educação Infantil": 
                    {
                       "Total": data["TotalI"],
                       "Creche": data["Creche"],
                       "Pré-Escola": data["Pré-Escola"]
                    },
                "Ensino Fundamental":
                    {
                        "Total": data["TotalF"],
                        "Anos Iniciais": data["Anos Iniciais"],
                        "Anos Finais": data["Anos Finais"]
                    },
                "Ensino Médio": data["Ensino Médio"],
                "Educação Profissional": data["Educação Profissional"],
                "Educação de Jovens e Adultos (EJA)": data["Educação de Jovens e Adultos (EJA)"],
                "Educação Especial": data["Educação Especial"]
            }};

    return dataJSON;

def filterPelotas (dataframe):
    filter = dataframe["Nome do Município"] == "Pelotas"
    
    filterPelotas = dataframe[filter]
    filterPelotas.reset_index(drop=True, inplace=True)
    
    return filterPelotas

def insertIntoDB (dataframe, collection):
    for index, row in dataframe.iterrows():
        teste1 = JSONstructure(row);

        collection.update_one({"Código INEP": row["Código da Escola"]}, {"$set": teste1});

### Conexão com o BD:

In [7]:
collection = openDataBaseMongoDB()

### Extração dos dados

In [8]:
higherEducation = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\PercentualDocentesComCursoSuperior.xlsx')

In [9]:
higherEducation.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,Creche,Pré-Escola,TotalF,Anos Iniciais,Anos Finais,Ensino Médio,Educação Profissional,Educação de Jovens e Adultos (EJA),Educação Especial
0,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
1,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN
2,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,NaN,NaN,NaN,100.0,NaN,100.0,NaN,NaN,NaN,NaN
3,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,100.0,NaN,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
4,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,100.0,NaN,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN


In [10]:
higherEducation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180331 entries, 0 to 180330
Data columns (total 19 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Ano                                 180331 non-null  int64  
 1   Região                              180331 non-null  object 
 2   UF                                  180331 non-null  object 
 3   Código do Município                 180331 non-null  int64  
 4   Nome do Município                   180331 non-null  object 
 5   Código da Escola                    180331 non-null  int64  
 6   Nome da Escola                      180331 non-null  object 
 7   Localização                         180331 non-null  object 
 8   Dependência Administrativa          180331 non-null  object 
 9   TotalI                              100558 non-null  float64
 10  Creche                              59059 non-null   float64
 11  Pré-Escola                

### Transformação dos dados

* Filtro das escolas de Pelotas

In [11]:
higherEducation_Pelotas = filterPelotas(higherEducation)

In [12]:
higherEducation_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalI,Creche,Pré-Escola,TotalF,Anos Iniciais,Anos Finais,Ensino Médio,Educação Profissional,Educação de Jovens e Adultos (EJA),Educação Especial
0,2019,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN
1,2019,Sul,RS,4314407,Pelotas,43001165,EEI CRIANCA AMADA,Urbana,Privada,30.0,14.3,66.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Sul,RS,4314407,Pelotas,43001173,EEI MUNDO DO MICKEY,Urbana,Privada,100.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,Sul,RS,4314407,Pelotas,43002145,EEI CHAPEUZINHO VERMELHO,Urbana,Privada,14.3,33.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,NaN,NaN,NaN,97.6,96.9,100.0,NaN,NaN,NaN,NaN


higherEducation_Pelotas.info()

* Colunas desnecessárias

In [13]:
deletingColumns(higherEducation_Pelotas)

In [14]:
higherEducation_Pelotas.head()

,Ano,Código da Escola,TotalI,Creche,Pré-Escola,TotalF,Anos Iniciais,Anos Finais,Ensino Médio,Educação Profissional,Educação de Jovens e Adultos (EJA),Educação Especial
0,2019,43000304,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN
1,2019,43001165,30.0,14.3,66.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,43001173,100.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,43002145,14.3,33.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,43003222,NaN,NaN,NaN,97.6,96.9,100.0,NaN,NaN,NaN,NaN


### Teste JSON

In [15]:
rowData = higherEducation_Pelotas.iloc[ 0 , : ]

In [16]:
JSONstructure(rowData)

{'Percentual de funções docentes com curso superior': {'Última atualização': 2019.0,
  'Educação Infantil': {'Total': nan, 'Creche': nan, 'Pré-Escola': nan},
  'Ensino Fundamental': {'Total': nan,
   'Anos Iniciais': nan,
   'Anos Finais': nan},
  'Ensino Médio': 100.0,
  'Educação Profissional': nan,
  'Educação de Jovens e Adultos (EJA)': nan,
  'Educação Especial': nan}}

### Carregamento dos dados

* Inserindo no Banco de Dados (MongoDB)

In [17]:
insertIntoDB(higherEducation_Pelotas, collection)

In [18]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2620d181e7e7953bdb84af'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Média de alunos por turma': {'Última atualização': 2019.0,
  'Educação Infantil': {'Total': nan, 'Creche': nan